In [54]:
import numpy as np
import plotly.graph_objects as go
import notebooks.latex_utils as latex

# P = np.array([ [x, np.sqrt(x)] for x in np.linspace(0, 100, 3)])
P = np.array([
    [0, 0],
    [1, 1],
    [3, -6],
    [4, -2.5],
    [5, 3],
    [6, -0.8],
    [7, 2],
    [9, 1.5],
    [10, -0.5]
])

n = P.shape[0] - 1

X = P[:, 0]
Y = P[:, 1]
print(f"X = {X.shape} {X}")
print(f"Y = {Y}")

tau = 0.01
tau = 0.001
# tau = 10

h = np.zeros(n)
for i in range(n):
    h[i] = X[i + 1] - X[i]

print(f"h = {h}")

alpha = np.zeros(n)
for i in range(n):
    alpha[i] = 1 / h[i] - tau / np.sinh(tau * h[i])
print(f"alpha = {alpha}")

beta = np.zeros(n)
for i in range(n):
    beta[i] = tau * (1.0 / np.tanh(tau * h[i])) - 1 / h[i]
print(f"beta = {beta}")

gamma = np.zeros(n)
for i in range(n):
    gamma[i] = tau ** 2 / h[i] * (Y[i + 1] - Y[i])
print(f"gamma = {gamma}")

# v = np.zeros(n + 1)
# for i in range(1, n):
#     v[i] = gamma[i] - gamma[i - 1]
# print(f"v = {v}")
#
# b = np.zeros(n + 1)
# for i in range(1, n):
#     b[i] = beta[i] + beta[i - 1]
# print(f"b = {b}")

C = np.zeros(n - 1)
for i in range(n-1):
    C[i] = gamma[i+1] - gamma[i]
A = np.zeros((n - 1, n-1))

for i in range(n-1):
    if i == 0:
        A[i, i] = beta[i] + beta[i+1]
    elif i == n-2:
        A[i, i-1] = alpha[n-2]
        A[i, i] = beta[n-2] + beta[n-1]
    else:
        A[i, i-1] = alpha[i]
        A[i, i] = beta[i] + beta[i+1]
        A[i, i+1] = alpha[i+1]

z = np.concatenate(([0], np.linalg.solve(A, C), [0]))

latex.print_latex(latex.matrix(A) + "\cdot z = " + latex.matrix(np.array([C])))

# z = np.zeros(n + 1)
# z[n] = 0
# z[0] = 0
# z[n - 1] = (gamma[n - 1] - gamma[n - 2]) / alpha[n - 1]
#
# for i in range(n - 2, 0, -1):
#     z[i] = 1 / alpha[i] * (v[i] - 1 / alpha[i + 1] * b[i - 1] * v[i - 1])
print(f"z = {z}")

tau2 = tau ** 2


def S(x):
    n = z.size - 1
    i = 0
    for j in range(n, -1, -1):
        if x - X[j] >= 0:
            i = j
            break
    if i >= n:
        i = n - 1
    return (z[i] * np.sinh(tau * (X[i + 1] - x)) + z[i + 1] * np.sinh(tau * (x - X[i]))) / (tau2 * np.sinh(tau * h[i])) + \
        (Y[i] - z[i]/tau2) * (X[i+1] - x) / h[i] + (Y[i+1] - z[i+1]/tau2)* (x - X[i])/h[i]


fig1 = go.Scatter(x=X, y=Y, name='Data points')
fig2 = go.Scatter(x=X, y=np.zeros(P.shape[0]), mode='markers', line=dict(color='red'))
XX = np.linspace(X[0], X[n], 1000)
YY = [S(x) for x in XX]
fig3 = go.Scatter(x=XX, y=YY, line=dict(color='white'), name='Interpolation')

fig = go.Figure(fig1)
fig.add_trace(fig2)
fig.add_trace(fig3)
fig.show()


X = (9,) [ 0.  1.  3.  4.  5.  6.  7.  9. 10.]
Y = [ 0.   1.  -6.  -2.5  3.  -0.8  2.   1.5 -0.5]
h = [1. 2. 1. 1. 1. 1. 2. 1.]
alpha = [1.66666647e-07 3.33333178e-07 1.66666647e-07 1.66666647e-07
 1.66666647e-07 1.66666647e-07 3.33333178e-07 1.66666647e-07]
beta = [3.33333311e-07 6.66666489e-07 3.33333311e-07 3.33333311e-07
 3.33333311e-07 3.33333311e-07 6.66666489e-07 3.33333311e-07]
gamma = [ 1.0e-06 -3.5e-06  3.5e-06  5.5e-06 -3.8e-06  2.8e-06 -2.5e-07 -2.0e-06]


$$\begin{align}\begin{bmatrix}9.99999800077589e-07&0.0&0.0&0.0&0.0&0.0&0.0\\3.3333317783768024e-07&9.99999800077589e-07&1.6666664726106006e-07&0.0&0.0&0.0&0.0\\0.0&1.6666664726106006e-07&6.66666622350931e-07&1.6666664726106006e-07&0.0&0.0&0.0\\0.0&0.0&1.6666664726106006e-07&6.66666622350931e-07&1.6666664726106006e-07&0.0&0.0\\0.0&0.0&0.0&1.6666664726106006e-07&6.66666622350931e-07&1.6666664726106006e-07&0.0\\0.0&0.0&0.0&0.0&1.6666664726106006e-07&9.99999800077589e-07&3.3333317783768024e-07\\0.0&0.0&0.0&0.0&0.0&3.3333317783768024e-07&9.99999800077589e-07\\\end{bmatrix}\cdot z = \begin{bmatrix}-4.5e-06&7e-06&2e-06&-9.299999999999999e-06&6.5999999999999995e-06&-3.05e-06&-1.75e-06\\\end{bmatrix}\end{align}$$

z = [  0.          -4.5000009    7.49963222   6.00221397 -19.5084879
  16.23173505  -5.81845092   0.18948277   0.        ]
